# Temporal Graphs and Path Data

## Motivation and Learning Objectives

In this tutorial we will introduce the representation of temporal graph data in the `Temporal Graph` class and how such data can be used to calculate time respecting paths.


In [1]:
import torch
from torch_geometric.data import TemporalData
from torch_geometric.utils import remove_isolated_nodes
import numpy as np
import pathpyG as pp

pp.config['torch']['device'] = 'cpu'

In [2]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.N)
print(t.M)

9
20


In [3]:
print(t.data)

TemporalData(src=[20], dst=[20], t=[20])


In [4]:
td = TemporalData(
    src = torch.Tensor([0,1,2,0]),
    dst = torch.Tensor([1,2,3,1]), 
    t = torch.Tensor([0,1,2,3]))
print(td)

TemporalData(src=[4], dst=[4], t=[4])


In [5]:
t2 = pp.TemporalGraph(td)

In [6]:
t1 = t.get_window(0,4)
print(t1)
print(t1.N)
print(t1.M)

Temporal Graph with 5 nodes, 4 unique edges and 4 events in [1.0, 9.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([4])
	t		<class 'torch.Tensor'> -> torch.Size([4])
	src		<class 'torch.Tensor'> -> torch.Size([4])

5
4


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [7]:
g = t.to_static_graph()
print(g)

Graph with 9 nodes and 20 edges

Graph attributes
	mapping		<class 'pathpyG.core.IndexMap.IndexMap'>
	num_nodes		<class 'int'>



In [8]:
g = t.to_static_graph((1, 10))
print(g)

Graph with 9 nodes and 17 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([17])

Graph attributes
	num_nodes		<class 'int'>



In [9]:
r = pp.algorithms.RollingTimeWindow(t, 10, 10, return_window=True)
for g, w in r:
    print('Time window ', w)
    print(g)
    print(g.data.edge_index)
    print('---')

Time window  (1.0, 11.0)
Graph with 5 nodes and 4 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([4])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 1, 2, 2],
           [1, 2, 3, 4]], sparse_size=(3, ?), nnz=4, sort_order=row)
---
Time window  (11.0, 21.0)
Graph with 7 nodes and 3 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 2, 5],
           [6, 5, 0]], sparse_size=(6, ?), nnz=3, sort_order=row)
---
Time window  (21.0, 31.0)
Graph with 8 nodes and 6 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([6])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 0, 1, 2, 6, 7],
           [2, 6, 5, 5, 7, 5]], sparse_size=(8, ?), nnz=6, sort_order=row)
---
Time window  (31.0, 41.0)
Graph with 8 nodes and 3 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

E

## Temporal Graphs

Let's start with a simple temporal graph with four nodes `a`,`b`,`c`,`d` and seven timestamped edges `(b,c;2)`,`(a,b;1)`,`(c,d;3)`,`(d,a;4)`,`(b,d;2)`, `(d,a;6)`,`(a,b;7)`. 

The following code generates this temporal graph from the given edge list.

In [10]:
g = pp.TemporalGraph.from_edge_list([['b', 'c', 2],['a', 'b', 1], ['c', 'd', 3], ['d', 'a', 4], ['b', 'd', 2], ['d', 'a', 6], ['a', 'b', 7]])
print(g)

Temporal Graph with 4 nodes, 5 unique edges and 7 events in [1.0, 7.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([7])
	t		<class 'torch.Tensor'> -> torch.Size([7])
	src		<class 'torch.Tensor'> -> torch.Size([7])



We can visualize a temporal graph by using the pathpyG plot function.

In [11]:
pp.plot(g, edge_color='lightgray')

Consistent with `pyG` the sources, destinations and timestamps are stored as a `pyG TemporalData` object, which we can access in the following way.



In [12]:
g.data

TemporalData(src=[7], dst=[7], t=[7])

In [13]:
print(g.data.t)

tensor([1., 2., 2., 3., 4., 6., 7.])


With the generator functions `edges` and `temporal_edges` we can iterate through the (temporal) edges of this graph.

In [14]:
for v, w in g.edges:
    print(v, w)

a b
b c
b d
c d
d a
d a
a b


In [15]:
for v, w, t in g.temporal_edges:
    print(v, w, t)

a b 1.0
b c 2.0
b d 2.0
c d 3.0
d a 4.0
d a 6.0
a b 7.0


## Reading Temporal Graphs from CSV files

In [16]:
t = pp.TemporalGraph.from_csv('../data/ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([1911])
	t		<class 'torch.Tensor'> -> torch.Size([1911])
	src		<class 'torch.Tensor'> -> torch.Size([1911])



In [17]:
t = pp.TemporalGraph.from_csv('../data/manufacturing_email.tedges', time_rescale=20)
print(t)

Temporal Graph with 167 nodes, 5784 unique edges and 82927 events in [63122700.0, 64294224.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([82927])
	t		<class 'torch.Tensor'> -> torch.Size([82927])
	src		<class 'torch.Tensor'> -> torch.Size([82927])



## Extracting Causal Topologies via Node-Time Event DAGs

We are often interested in the time respecting paths of a temporal graph.

A time respecting path is defined as a sequence of nodes $v_0,...,v_l$ where the corresponding edges occur in the right time ordering and with a maximum time difference of $\delta\in \N$. 

In order to extract those paths out of a temporal graph, we have to construct a time-unfolded directed acyclic graph (DAG) that represents the network and captures all causal structures.

The nodes of a DAG are node-time-events of the temporal graph, i.e a node `a-t` represents the node `a` at time `t`. 

Two nodes `a-t` and `b-t'` are connected (with exceptions, see code ) if `a-t` directly influences `b-t'` 


In [18]:
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
print(dag)

print(dag.mapping)
print(dag.data.edge_index)

pp.plot(dag, edge_color='lightgray')

Graph with 9 nodes and 7 edges

Node attributes
	node_name		<class 'list'>
	node_idx		<class 'list'>

Edge attributes
	edge_ts		<class 'torch.Tensor'> -> torch.Size([7])

Graph attributes
	num_nodes		<class 'int'>
	temporal_graph_index_map		<class 'list'>

a-1.0 -> 0
b-2.0 -> 1
c-3.0 -> 2
d-3.0 -> 3
d-4.0 -> 4
a-5.0 -> 5
d-6.0 -> 6
a-7.0 -> 7
b-8.0 -> 8

EdgeIndex([[0, 1, 1, 2, 4, 6, 7],
           [1, 2, 3, 4, 5, 7, 8]], sparse_size=(8, ?), nnz=7, sort_order=row)


With the following code, we can extract DAGs with only one single root node, which is not influenced by any other node-time event in the temporal graph.

In [19]:
x = pp.algorithms.extract_causal_trees(dag)
print(x)

{'a-1.0': tensor([[0, 1, 1, 2, 4],
        [1, 2, 3, 4, 5]], dtype=torch.int32), 'd-6.0': tensor([[6, 7],
        [7, 8]], dtype=torch.int32)}


## Higher-Order De Bruijn Graph Models for Causal Paths

With the DAG, we can now extract the time-respecting paths in our temporal graph.

In [20]:
paths = pp.DAGData.from_temporal_dag(dag)
print(paths.paths[0])
print(paths.paths[1])
print(g.mapping)

tensor([[0, 1, 1, 2, 4],
        [1, 2, 3, 4, 5]])
tensor([[6, 7],
        [7, 8]])
b -> 0
c -> 1
a -> 2
d -> 3



The path data allows us to construct a Higher-Order De Bruijn Graph model belong to our temporal network.

In [21]:
g2 = pp.HigherOrderGraph(paths, order=2, node_ids=g.mapping.node_ids)
pp.plot(g2)

In a nutshell, the following shows the steps that are needed to construct a Higher Order Graph. Here we construct our temporal graph from a `pyG TemporalData` by providing the sources, distances and timestamps.

In [22]:
# Create temporal network
data = TemporalData(src=torch.LongTensor([0,2,1,2,0,2,1,2]), dst=torch.LongTensor([2,3,2,4,2,3,2,4]), t=torch.LongTensor([1,2,3,4,5,6,7,8]))
g = pp.TemporalGraph(data, mapping=pp.IndexMap(['a', 'b', 'c', 'd', 'e']))

# Create event DAG and extract path data
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
paths = pp.DAGData.from_temporal_dag(dag)

# Create Higher Order Graph
g2 = pp.HigherOrderGraph(paths, order=2)
pp.plot(g2)

We can also skip the step of creating an event DAG by just using the following code.

In [25]:
g2 = pp.HigherOrderGraph.from_temporal_graph(g, delta=1, order=2)
pp.plot(g2);